In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip -q install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.5 MB/s eta 0:00:00


In [4]:
# ============================================================
# YOLO(ultralytics) + ResNet18 분류모델
# - validation/00-damage(파손) / validation/01-whole(정상) 검증
# - 각 모델별 Accuracy, F1, Confusion Matrix
# - (정상/파손)별로 "정상추론 10개" + "오추론 10개" 시각화
# ============================================================

# ---- (0) 설치/임포트 ----
# Colab에서 ultralytics가 없으면 아래 설치가 필요합니다.
# !pip -q install ultralytics

import os
import random
from pathlib import Path

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import models, transforms

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# YOLO
from ultralytics import YOLO


# ---- (1) 경로 설정 ----
YOLO_WEIGHT = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/(25.12.24)_yolo_best_weight.pt")
RESNET_WEIGHT = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/(share)HDMF_AUTO_SPOKE/SUBJECT/WEEK2_CAR_DAMAGE_DETECTION/JR/01.Model/(25.12.24)_resnet18_best.pt")

VAL_DAMAGE_DIR = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/00. Datasets/Kaggle(Car damage detection)/validation/00-damage")
VAL_WHOLE_DIR  = Path("/content/gdrive/MyDrive/01.DS Part/99.Study/01.Vehicle_Damage_Detection/00. Datasets/Kaggle(Car damage detection)/validation/01-whole")

assert YOLO_WEIGHT.exists(), f"YOLO weight not found: {YOLO_WEIGHT}"
assert RESNET_WEIGHT.exists(), f"ResNet weight not found: {RESNET_WEIGHT}"
assert VAL_DAMAGE_DIR.exists(), f"Validation damage dir not found: {VAL_DAMAGE_DIR}"
assert VAL_WHOLE_DIR.exists(),  f"Validation whole dir not found: {VAL_WHOLE_DIR}"

# 라벨 정의 (통일)
# 0 = damage, 1 = whole
LABEL_MAP = {"damage": 0, "whole": 1}
INV_LABEL = {0: "damage", 1: "whole"}


# ---- (2) 유틸: 이미지 파일 수집 ----
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

def list_images(folder: Path):
    paths = []
    for p in folder.rglob("*"):
        if p.is_file() and p.suffix.lower() in IMG_EXTS:
            paths.append(p)
    return sorted(paths)

damage_imgs = list_images(VAL_DAMAGE_DIR)
whole_imgs  = list_images(VAL_WHOLE_DIR)

print(f"[INFO] damage images: {len(damage_imgs)}")
print(f"[INFO] whole  images: {len(whole_imgs)}")

all_paths = damage_imgs + whole_imgs
all_y_true = ([LABEL_MAP["damage"]] * len(damage_imgs)) + ([LABEL_MAP["whole"]] * len(whole_imgs))


# ---- (3) 디바이스 ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] device = {device}")


# ---- (4) ResNet18 로더 (binary classifier 가정) ----
# ⚠️ 주의:
# - 학습 때 마지막 레이어를 (num_classes=2)로 바꿔 저장했다면 아래 코드로 로드됩니다.
# - 만약 state_dict 저장 시 키가 다르거나, FC 구조가 다르면 에러가 날 수 있습니다.
#   그 경우 출력되는 에러 메시지 기준으로 fc 이름/차원을 맞춰주면 됩니다.

def load_resnet18_binary(weight_path: Path, device):
    model = models.resnet18(weights=None)   # 가중치 파일을 직접 로드하므로 weights=None
    model.fc = nn.Linear(model.fc.in_features, 2)
    ckpt = torch.load(weight_path, map_location="cpu")

    # state_dict 형태 다양성 대응
    if isinstance(ckpt, dict) and "state_dict" in ckpt:
        state_dict = ckpt["state_dict"]
    elif isinstance(ckpt, dict) and "model_state_dict" in ckpt:
        state_dict = ckpt["model_state_dict"]
    elif isinstance(ckpt, dict) and any(k.startswith("module.") for k in ckpt.keys()):
        state_dict = ckpt
    else:
        state_dict = ckpt

    # module. prefix 제거(DDP/DP 저장 대비)
    new_state = {}
    for k, v in state_dict.items():
        nk = k.replace("module.", "")
        new_state[nk] = v

    model.load_state_dict(new_state, strict=True)
    model.eval().to(device)
    return model

resnet = load_resnet18_binary(RESNET_WEIGHT, device)

# ResNet 전처리 (ImageNet 기준)
resnet_tf = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])


# ---- (5) YOLO 로더 ----
yolo = YOLO(str(YOLO_WEIGHT))


# ---- (6) 예측 함수들 ----
@torch.no_grad()
def predict_resnet_batch(paths, model, device, batch_size=64):
    preds = []
    for i in range(0, len(paths), batch_size):
        batch_paths = paths[i:i+batch_size]
        imgs = []
        for p in batch_paths:
            img = Image.open(p).convert("RGB")
            imgs.append(resnet_tf(img))
        x = torch.stack(imgs, dim=0).to(device)
        logits = model(x)
        pred = torch.argmax(logits, dim=1).detach().cpu().numpy().tolist()
        preds.extend(pred)
    return preds


def predict_yolo(paths, yolo_model, imgsz=640, conf=0.25, device_str=None):
    """
    Ultralytics YOLO 'classify' head로 학습한 모델을 가정합니다.
    - 결과에서 top1 클래스(정수)를 꺼내 예측으로 사용합니다.
    """
    preds = []
    # device_str: "0" or "cpu" 등. None이면 ultralytics가 자동 선택
    results = yolo_model.predict(
        source=[str(p) for p in paths],
        imgsz=imgsz,
        conf=conf,
        verbose=False,
        device=device_str
    )
    for r in results:
        # classify model의 경우: r.probs.top1 제공
        if hasattr(r, "probs") and r.probs is not None:
            preds.append(int(r.probs.top1))
        else:
            raise RuntimeError(
                "YOLO 결과에서 r.probs를 찾지 못했습니다. "
                "이 가중치가 'classification' 모델이 아닌지 확인해 주세요."
            )
    return preds


# ---- (7) 평가/리포트 함수 ----
def evaluate_and_report(y_true, y_pred, title="Model"):
    acc = accuracy_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred, average="binary", pos_label=LABEL_MAP["damage"])  # damage를 positive로
    cm  = confusion_matrix(y_true, y_pred, labels=[0,1])
    print("="*70)
    print(f"[{title}]")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 (pos=damage): {f1:.4f}")
    print("Confusion Matrix (rows=true, cols=pred) labels=[damage(0), whole(1)]")
    print(cm)
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, target_names=["damage(0)", "whole(1)"]))
    return acc, f1, cm


# ---- (8) 시각화(그리드) ----
def sample_paths(paths, k=10, seed=42):
    if len(paths) == 0:
        return []
    rng = random.Random(seed)
    if len(paths) <= k:
        return paths
    return rng.sample(paths, k)

def show_image_grid(paths, title, ncols=5, figsize=(16, 7)):
    if len(paths) == 0:
        print(f"[WARN] {title}: 표시할 이미지가 없습니다.")
        return
    n = len(paths)
    ncols = min(ncols, n)
    nrows = int(np.ceil(n / ncols))
    plt.figure(figsize=figsize)
    for i, p in enumerate(paths):
        ax = plt.subplot(nrows, ncols, i+1)
        img = Image.open(p).convert("RGB")
        ax.imshow(img)
        ax.set_title(p.name, fontsize=9)
        ax.axis("off")
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

def make_correct_wrong_sets(paths, y_true, y_pred):
    """
    (정상/파손) 별로
    - 정상추론(correct)
    - 오추론(wrong)
    리스트를 반환
    """
    damage_correct, damage_wrong = [], []
    whole_correct, whole_wrong   = [], []

    for p, yt, yp in zip(paths, y_true, y_pred):
        if yt == LABEL_MAP["damage"]:
            if yp == yt:
                damage_correct.append(p)
            else:
                damage_wrong.append(p)
        else:  # whole
            if yp == yt:
                whole_correct.append(p)
            else:
                whole_wrong.append(p)
    return damage_correct, damage_wrong, whole_correct, whole_wrong


# ---- (9) 실행: ResNet 평가 + 시각화 ----
print("\n\n[RUN] ResNet18 inference...")
resnet_pred = predict_resnet_batch(all_paths, resnet, device, batch_size=64)
evaluate_and_report(all_y_true, resnet_pred, title="ResNet18 (best.pt)")

d_c, d_w, w_c, w_w = make_correct_wrong_sets(all_paths, all_y_true, resnet_pred)

show_image_grid(sample_paths(d_c, 10, seed=1), "ResNet - DAMAGE(파손) correct (up to 10)")
show_image_grid(sample_paths(d_w, 10, seed=2), "ResNet - DAMAGE(파손) wrong (up to 10)")
show_image_grid(sample_paths(w_c, 10, seed=3), "ResNet - WHOLE(정상) correct (up to 10)")
show_image_grid(sample_paths(w_w, 10, seed=4), "ResNet - WHOLE(정상) wrong (up to 10)")


# ---- (10) 실행: YOLO 평가 + 시각화 ----
print("\n\n[RUN] YOLO inference...")
# ultralytics 내부에서 cuda를 잘 잡지만, 필요하면 device_str="0" 또는 "cpu"로 고정 가능
device_str = "0" if torch.cuda.is_available() else "cpu"

yolo_pred = predict_yolo(all_paths, yolo, imgsz=640, conf=0.25, device_str=device_str)
evaluate_and_report(all_y_true, yolo_pred, title="YOLO (best_weight.pt)")

d_c, d_w, w_c, w_w = make_correct_wrong_sets(all_paths, all_y_true, yolo_pred)

show_image_grid(sample_paths(d_c, 10, seed=11), "YOLO - DAMAGE(파손) correct (up to 10)")
show_image_grid(sample_paths(d_w, 10, seed=12), "YOLO - DAMAGE(파손) wrong (up to 10)")
show_image_grid(sample_paths(w_c, 10, seed=13), "YOLO - WHOLE(정상) correct (up to 10)")
show_image_grid(sample_paths(w_w, 10, seed=14), "YOLO - WHOLE(정상) wrong (up to 10)")


# ============================================================
# (옵션) 오추론 파일명을 따로 보고 싶으면 아래를 실행하세요.
# ============================================================
def print_misclassified(paths, y_true, y_pred, max_show=30):
    mis = []
    for p, yt, yp in zip(paths, y_true, y_pred):
        if yt != yp:
            mis.append((p, yt, yp))
    print(f"[INFO] misclassified = {len(mis)}")
    for i, (p, yt, yp) in enumerate(mis[:max_show]):
        print(f"{i+1:02d}. {p} | true={INV_LABEL[yt]} pred={INV_LABEL[yp]}")
    return mis

# 예: ResNet 오추론 목록
# _ = print_misclassified(all_paths, all_y_true, resnet_pred, max_show=50)

# 예: YOLO 오추론 목록
# _ = print_misclassified(all_paths, all_y_true, yolo_pred, max_show=50)


Output hidden; open in https://colab.research.google.com to view.